In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'train.csv', 'test.csv', 'embeddings']


In [2]:
%%time
train_df=pd.read_csv("../input/train.csv")
test=pd.read_csv("../input/test.csv")

CPU times: user 2.58 s, sys: 296 ms, total: 2.88 s
Wall time: 2.89 s


In [3]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D,CuDNNLSTM, Bidirectional,GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D,BatchNormalization,GaussianDropout,GaussianNoise
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.engine.topology import Layer
from sklearn.metrics import f1_score
from keras import backend as K
from keras.initializers import *
from keras.optimizers import *
from keras.layers import concatenate
import tensorflow as tf
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(tf.keras.backend.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = tf.keras.backend.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)
def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale


Using TensorFlow backend.


In [4]:
# %%time
# #baseline 0.6767962042476277
# #np.where(pred>=0.5,1,0)
# from keras.preprocessing import sequence
# from keras.utils import np_utils
# from keras.models import Sequential
# from keras.preprocessing.text import Tokenizer
# from sklearn.model_selection import train_test_split
# from keras.preprocessing.sequence import pad_sequences
# from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D,CuDNNLSTM, Bidirectional,GlobalMaxPool1D
# from keras.models import Model
# train, val = train_test_split(train_df, test_size = 0.2, random_state=100)
# embed_size = 300 # how big is each word vector
# max_features = 60000 # how many unique words to use (i.e num rows in embedding vector)
# maxlen = 110 # max number of words in a question to use
# tokenizer = Tokenizer(num_words=max_features)
# tokenizer.fit_on_texts(list(train['question_text']))
# train_x = tokenizer.texts_to_sequences(train['question_text'])
# val_x = tokenizer.texts_to_sequences(val['question_text'])
# train_x=pad_sequences(train_x,maxlen=maxlen)
# val_x=pad_sequences(val_x,maxlen=maxlen)
# train_y=train['target'].values
# val_y=val['target'].values
# word_index = tokenizer.word_index

# def embed(inp,max_features=max_features,word_index=word_index):
#     EMBEDDING_FILE = inp
#     def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
#     embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if o.split(" ")[0] in word_index)
#     all_embs = np.stack(embeddings_index.values())
#     emb_mean,emb_std = all_embs.mean(), all_embs.std()
#     embed_size = all_embs.shape[1]
#     nb_words = min(max_features, len(word_index))
#     embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
#     for word, i in word_index.items():
#         if i >= max_features: continue
#         embedding_vector = embeddings_index.get(word)
#         if embedding_vector is not None: embedding_matrix[i] = embedding_vector
#     return embedding_matrix

# embedding_matrix=embed('../input/embeddings/glove.840B.300d/glove.840B.300d.txt')
# #del embedding_matrix_para,embedding_matrix_glove
# inp=Input(shape=(maxlen,))
# x=Embedding(max_features,embed_size,weights=[embedding_matrix])(inp)
# x=Bidirectional(CuDNNLSTM(64,return_sequences=True))(x)
# x=GlobalMaxPool1D()(x)
# x=Dense(16, activation="relu")(x)
# x=Dropout(0.1)(x)
# x=Dense(1,activation='sigmoid')(x)
# model=Model(inputs=inp,outputs=x)
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=[f1])
# model.fit(train_x, train_y, batch_size=512, epochs=2, validation_data=(val_x, val_y))

In [5]:
# from sklearn import metrics
# pred_val=model.predict([val_x], batch_size=1024, verbose=1)
# test_x = tokenizer.texts_to_sequences(test['question_text'])
# test_x=pad_sequences(test_x,maxlen=maxlen)
# pred_test=model.predict([test_x], batch_size=1024, verbose=1)
# prediction=pd.DataFrame(pred_test,columns=['baseline'])
# data_x = tokenizer.texts_to_sequences(train_df['question_text'])
# data_x=pad_sequences(data_x,maxlen=maxlen)
# pred_data=model.predict([data_x], batch_size=1024, verbose=1)
# main_data=pd.DataFrame(pred_val,columns=['baseline'])
# thresholds = []
# for thresh in np.arange(0.1, 0.501, 0.01):  
#     thresh = np.round(thresh, 2)
#     res = metrics.f1_score(val_y, (pred_val > thresh).astype(int))
#     thresholds.append([thresh, res])
# thresholds.sort(key=lambda x: x[1], reverse=True)
# best_thresh = thresholds[0][0]
# metrics.f1_score(val_y, (pred_val > best_thresh).astype(int))

In [6]:
%%time
#result 0.676764822246086
#np.shape(np.stack((pred_val_gl,pred_val_wiki,pred_val_para)))
#pd.DataFrame({'gl':pred_val_gl,'wiki':pred_val_wiki,'para':pred_val_para},index='gl')
#baseline 0.6775440857695236
#np.where(pred>=0.5,1,0)

# train, val = train_test_split(train_df, test_size = 0.2, random_state=100)
# embed_size = 300 # how big is each word vector
# max_features = 100000 # how many unique words to use (i.e num rows in embedding vector)
# maxlen = 110 # max number of words in a question to use
# tokenizer = Tokenizer(num_words=max_features)
# tokenizer.fit_on_texts(list(train['question_text']))
# train_x = tokenizer.texts_to_sequences(train['question_text'])
# val_x = tokenizer.texts_to_sequences(val['question_text'])
# train_x=pad_sequences(train_x,maxlen=maxlen)
# val_x=pad_sequences(val_x,maxlen=maxlen)
# train_y=train['target'].values
# val_y=val['target'].values
# word_index = tokenizer.word_index

# def embed(inp,max_features=max_features,word_index=word_index):
#     EMBEDDING_FILE = inp
#     def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
#     embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if o.split(" ")[0] in word_index)
#     all_embs = np.stack(embeddings_index.values())
#     emb_mean,emb_std = all_embs.mean(), all_embs.std()
#     embed_size = all_embs.shape[1]
#     nb_words = min(max_features, len(word_index))
#     embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
#     for word, i in word_index.items():
#         if i >= max_features: continue
#         embedding_vector = embeddings_index.get(word)
#         if embedding_vector is not None: embedding_matrix[i] = embedding_vector
#     return embedding_matrix

# embedding_matrix=embed('../input/embeddings/glove.840B.300d/glove.840B.300d.txt')
# inp = Input(shape=(maxlen,))
# x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
# x = SpatialDropout1D(0.1)(x)
# x = Bidirectional(CuDNNLSTM(40, return_sequences=True))(x)
# y = Bidirectional(CuDNNGRU(40, return_sequences=True))(x)

# atten_1 = Attention(maxlen)(x) # skip connect
# atten_2 = Attention(maxlen)(y)
# avg_pool = GlobalAveragePooling1D()(y)
# max_pool = GlobalMaxPooling1D()(y)

# conc = concatenate([atten_1, atten_2, avg_pool, max_pool])
# conc = Dense(16, activation="relu")(conc)
# conc = Dropout(0.1)(conc)
# outp = Dense(1, activation="sigmoid")(conc)    

# model = Model(inputs=inp, outputs=outp)
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])
# model.fit(train_x, train_y, batch_size=512, epochs=5, validation_data=(val_x, val_y))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.82 µs


In [7]:
rare_words_mapping = {' s.p ': ' ', ' S.P ': ' ', 'U.s.p': '', 'U.S.A.': 'USA', 'u.s.a.': 'USA', 'U.S.A': 'USA',
                      'u.s.a': 'USA', 'U.S.': 'USA', 'u.s.': 'USA', ' U.S ': ' USA ', ' u.s ': ' USA ', 'U.s.': 'USA',
                      ' U.s ': 'USA', ' u.S ': ' USA ', 'fu.k': 'fuck', 'U.K.': 'UK', ' u.k ': ' UK ',
                      ' don t ': ' do not ', 'bacteries': 'batteries', ' yr old ': ' years old ', 'Ph.D': 'PhD',
                      'cau.sing': 'causing', 'Kim Jong-Un': 'The president of North Korea', 'savegely': 'savagely',
                      'Ra apist': 'Rapist', '2fifth': 'twenty fifth', '2third': 'twenty third',
                      '2nineth': 'twenty nineth', '2fourth': 'twenty fourth', '#metoo': 'MeToo',
                      'Trumpcare': 'Trump health care system', '4fifth': 'forty fifth', 'Remainers': 'remainder',
                      'Terroristan': 'terrorist', 'antibrahmin': 'anti brahmin',
                      'fuckboys': 'fuckboy', 'Fuckboys': 'fuckboy', 'Fuckboy': 'fuckboy', 'fuckgirls': 'fuck girls',
                      'fuckgirl': 'fuck girl', 'Trumpsters': 'Trump supporters', '4sixth': 'forty sixth',
                      'culturr': 'culture',
                      'weatern': 'western', '4fourth': 'forty fourth', 'emiratis': 'emirates', 'trumpers': 'Trumpster',
                      'indans': 'indians', 'mastuburate': 'masturbate', 'f**k': 'fuck', 'F**k': 'fuck', 'F**K': 'fuck',
                      ' u r ': ' you are ', ' u ': ' you ', '操你妈': 'fuck your mother', 'e.g.': 'for example',
                      'i.e.': 'in other words', '...': '.', 'et.al': 'elsewhere', 'anti-Semitic': 'anti-semitic',
                      'f***': 'fuck', 'f**': 'fuc', 'F***': 'fuck', 'F**': 'fuc',
                      'a****': 'assho', 'a**': 'ass', 'h***': 'hole', 'A****': 'assho', 'A**': 'ass', 'H***': 'hole',
                      's***': 'shit', 's**': 'shi', 'S***': 'shit', 'S**': 'shi', 'Sh**': 'shit',
                      'p****': 'pussy', 'p*ssy': 'pussy', 'P****': 'pussy',
                      'p***': 'porn', 'p*rn': 'porn', 'P***': 'porn',
                      'st*up*id': 'stupid',
                      'd***': 'dick', 'di**': 'dick', 'h*ck': 'hack',
                      'b*tch': 'bitch', 'bi*ch': 'bitch', 'bit*h': 'bitch', 'bitc*': 'bitch', 'b****': 'bitch',
                      'b***': 'bitc', 'b**': 'bit', 'b*ll': 'bull'
                      }


def pre_clean_rare_words(text):
    for rare_word in rare_words_mapping:
        if rare_word in text:
            text = text.replace(rare_word, rare_words_mapping[rare_word])

    return text

In [8]:
%%time
# from sklearn import metrics
# pred_val=model.predict([val_x], batch_size=1024, verbose=1)
# test_x = tokenizer.texts_to_sequences(test['question_text'])
# test_x=pad_sequences(test_x,maxlen=maxlen)
# pred_test=model.predict([test_x], batch_size=1024, verbose=1)
# prediction['Capsule']=pred_test
# data_x = tokenizer.texts_to_sequences(train_df['question_text'])
# data_x=pad_sequences(data_x,maxlen=maxlen)
# #pred_data=model.predict([data_x], batch_size=1024, verbose=1)
# main_data['Capsule']=pred_val
# thresholds = []
# for thresh in np.arange(0.1, 0.501, 0.01):
#     thresh = np.round(thresh, 2)
#     res = metrics.f1_score(val_y, (pred_val > thresh).astype(int))
#     thresholds.append([thresh, res])
# thresholds.sort(key=lambda x: x[1], reverse=True)
# best_thresh = thresholds[0][0]
# metrics.f1_score(val_y, (pred_val > best_thresh).astype(int))
import re
import string
##train_df['question_text']=train_df['question_text'].str.cat(train_df['question_text'].apply(lambda x: ' '.join([w[1] for w in pos_tag([s for s in x.split(' ') if s])])),sep=' ')
#test['question_text']=test['question_text'].str.cat(test['question_text'].apply(lambda x: ' '.join([w[1] for w in pos_tag([s for s in x.split(' ') if s])])),sep=' ')
#train_df['new_data']=train_df['question_text'].apply(lambda x: ' '.join([w[1] for w in pos_tag([s for s in x.split(' ') if s])]))
#test['new_data']=test['question_text'].apply(lambda x: ' '.join([w[1] for w in pos_tag([s for s in x.split(' ') if s])]))


puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)
train_df["question_text"] = train_df["question_text"].apply(lambda x: replace_typical_misspell(x))
test["question_text"] = test["question_text"].apply(lambda x: replace_typical_misspell(x))
#train_data=proc(train_df)
#test_data=proc(test)

train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_text(x))
test["question_text"] = test["question_text"].apply(lambda x: clean_text(x))
    
    # Clean numbers
#train_df["question_text"] = train_df["question_text"].apply(lambda x: clean_numbers(x))
#test["question_text"] = test["question_text"].apply(lambda x: clean_numbers(x))

train_df["question_text"] = train_df["question_text"].str.lower()
test["question_text"] = test["question_text"].str.lower()

train_df["question_text"] = train_df["question_text"].apply(lambda x: pre_clean_rare_words(x))
test["question_text"] = test["question_text"].apply(lambda x: pre_clean_rare_words(x))

train_df["question_text"] = train_df["question_text"].fillna("_##_")
test["question_text"] = test["question_text"].fillna("_##_")

def clean_latex_tag(text):
    corr_t = []
    for t in text.split(" "):
        t = t.strip()
        if t != '':
            corr_t.append(t)
    text = ' '.join(corr_t)
    
    text = re.sub('(\[ math \]).+(\[ / math \])', 'mathematical formula', text)
    return text

train_df['question_text'] = train_df['question_text'].map(clean_latex_tag)
test['question_text'] = test['question_text'].map(clean_latex_tag)

CPU times: user 1min 45s, sys: 440 ms, total: 1min 46s
Wall time: 1min 46s


In [9]:
test_meta=pd.DataFrame()
val_meta=[]
y_meta=[]
score=[]
max_features = 100000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_df['question_text']))
word_index = tokenizer.word_index
def embed(inp,**kwargs):
    EMBEDDING_FILE = inp
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if o.split(" ")[0] in word_index)
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    return embedding_matrix
def embed2(inp,**kwargs):
    EMBEDDING_FILE = inp
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if o.split(" ")[0] in word_index)
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    return embedding_matrix
embedding_matrix_glove=embed('../input/embeddings/glove.840B.300d/glove.840B.300d.txt',max_features=max_features,word_index=word_index)
embedding_matrix_para=embed2('../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt',max_features=max_features,word_index=word_index)
embedding_matrix=(embedding_matrix_glove*0.8)+(embedding_matrix_para*0.2)
from sklearn.model_selection import StratifiedKFold
rskf=StratifiedKFold(5,shuffle=True,random_state=1000)
for idx,(train_index,val_index) in enumerate(rskf.split(train_df.drop(['target'],axis=1),train_df['target'])):
    if idx==4:
        break
    train=train_df.loc[train_index]
    val=train_df.loc[val_index]
    embed_size = 300 # how big is each word vector
    max_features = 100000 # how many unique words to use (i.e num rows in embedding vector)
    maxlen = 72 # max number of words in a question to use
    train_x = tokenizer.texts_to_sequences(train['question_text'])
    val_x = tokenizer.texts_to_sequences(val['question_text'])
    train_x=pad_sequences(train_x,maxlen=maxlen)
    val_x=pad_sequences(val_x,maxlen=maxlen)
    train_y=train['target'].values
    val_y=val['target'].values
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(CuDNNLSTM(90, return_sequences=True))(x)
    y = Bidirectional(CuDNNGRU(90, return_sequences=True))(x)
    
    atten_1 = Attention(maxlen)(x) # skip connect
    atten_2 = Attention(maxlen)(y)
    avg_pool = GlobalAveragePooling1D()(y)
    max_pool = GlobalMaxPooling1D()(y)

    conc = concatenate([atten_1, atten_2, avg_pool, max_pool])
    conc = Dense(42, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])
    model.fit(train_x, train_y, batch_size=512, epochs=5, validation_data=(val_x, val_y))
    from sklearn import metrics
    pred_val=model.predict(val_x, batch_size=1024, verbose=1)
    test_x = tokenizer.texts_to_sequences(test['question_text'])
    test_x=pad_sequences(test_x,maxlen=maxlen)
    pred_test=model.predict(test_x, batch_size=1024, verbose=1)
    test_meta[idx+4]=pred_test.reshape(len(pred_test),)
    val_meta=np.concatenate((val_meta,pred_val.reshape(len(pred_val),)),axis=0)
    y_meta=np.concatenate((y_meta,val_y),axis=0)
    #prediction['Attention']=pred_test
    #pred_data=model.predict([data_x], batch_size=1024, verbose=1)
    #main_data['Attention']=pred_val
    from sklearn.metrics import roc_curve, precision_recall_curve
    def threshold_search(y_true, y_proba):
        precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
        thresholds = np.append(thresholds, 1.001) 
        F = 2 / (1/precision + 1/recall)
        best_score = np.max(F)
        best_th = thresholds[np.argmax(F)]
        search_result = {'threshold': best_th , 'f1': best_score}
        return search_result 
    score.append(threshold_search(val_y, pred_val)['threshold'])
pred_test=(test_meta.mean(axis=1).values>np.mean(score)).astype(int)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


Train on 1044897 samples, validate on 261225 samples
Epoch 1/5
1044897/1044897 [==============================] - 336s 322us/step - loss: 0.1201 - f1: 0.5479 - val_loss: 0.1034 - val_f1: 0.6305
Epoch 2/5
1044897/1044897 [==============================] - 333s 318us/step - loss: 0.1030 - f1: 0.6343 - val_loss: 0.0981 - val_f1: 0.6462
Epoch 3/5
 958464/1044897 [==========================>...] - ETA: 25s - loss: 0.0972 - f1: 0.6551

In [10]:
# train, val = train_test_split(main_data, test_size = 0.2, random_state=123)
# train_x=train.drop(['target'],axis=1)
# train_y=train['target']
# val_x=val.drop(['target'],axis=1)
# val_y=val['target']
# params = {
#     "objective" : "binary",
#     "metric" : "binary_error", 
#     "learning_rate" : 0.24,
#     'n_estimators': int(699.76),
#      'max_depth':int(round(3.4)),
#      'num_leaves':int(48.17),
#      'lambda_l1':max(1.18, 0),
#      'lambda_l2':max(2.82, 0),
#      'min_split_gain':0.0923,
#      'min_child_weight':5.41,
#      'bagging_fraction':max(min(0.961, 1), 0)
# }

# lgtrain = lgb.Dataset(train_x, label=train_y)
# lgval = lgb.Dataset(val_x, label=val_y)
# model = lgb.train(params, lgtrain, valid_sets=[lgval], early_stopping_rounds=80, verbose_eval=False)
# pred_val=model.predict(val_x, num_iteration=model.best_iteration)
# pred_test=model.predict(prediction, num_iteration=model.best_iteration)
# thresholds = []
# for thresh in np.arange(0.1, 0.501, 0.01):
#     thresh = np.round(thresh, 2)
#     res = metrics.f1_score(val_y, (pred_val > thresh).astype(int))
#     thresholds.append([thresh, res])
# thresholds.sort(key=lambda x: x[1], reverse=True)
# best_thresh = thresholds[0][0]
# pred_test=(pred_test>best_thresh).astype(int)
# metrics.f1_score(val_y, (pred_val > best_thresh).astype(int))
metrics.f1_score(y_meta,(val_meta>np.mean(score)).astype(int))

0.6880877404105986

In [11]:
#pred_test=(test_meta.mean(axis=1).values>np.mean(score)).astype(int)

In [12]:
#submission
out_df = pd.DataFrame({"qid":test["qid"].values})
out_df['prediction'] = pred_test
out_df.to_csv("submission.csv", index=False)